In [1]:
import os
class Config:
    # 数据加载器配置（保持与基础配置兼容，仅使用LibriSpeech开发集）
    DEBUG_MODE = True
    DEBUG_SAMPLE_SIZE = 5000
    BASE_DIR = os.getcwd()
    LIBRISPEECH_PATH = os.path.join(BASE_DIR, "devDataset", "LibriSpeech")  # 仅使用LibriSpeech开发集
    SAMPLE_RATE = 16000
    DURATION = 1.0
    MAX_SAMPLES = int(SAMPLE_RATE * DURATION)
    NOISE_TYPES = ["white", "babble"]  # 噪声类型适配复杂损失的鲁棒性训练
    SNR_LEVELS = [-5, 0, 5, 10]  # 扩展SNR范围以增强复杂损失的稳定性约束
    BATCH_SIZE = 16  # 适配复杂损失计算的内存需求
    NUM_WORKERS = 2
    VALID_RATIO = 0.1  # 从开发集中划分验证集
    MAX_SEQ_LEN = 16000
    
    # 模型配置（简化模型基础上适配复杂损失）
    INPUT_DIM = 1
    HIDDEN_DIM = 256
    EMBEDDING_DIM = 128  # 嵌入向量维度，用于复杂损失计算
    CHAOS_DIM = 64  # 混沌模块维度，影响相位同步损失
    CHAOS_TIME_STEPS = 5  # 混沌时间步，与李雅普诺夫损失相关
    ATTENTION_HEADS = 4
    DROPOUT_RATE = 0.1  # 新增dropout增强泛化性
    
    # 复杂损失配置（T5模型核心扩展）
    CE_WEIGHT = 1.0  # 交叉熵损失权重
    SYNC_WEIGHT = 0.3  # 相位同步损失权重
    LYAPUNOV_WEIGHT = 0.2  # 李雅普诺夫稳定性损失权重
    LOSS_WARMUP_EPOCHS = 3  # 损失预热周期，逐步启用复杂损失
    
    # 训练配置（针对多损失联合优化调整）
    EPOCHS = 500
    LR = 0.005  # 略低于基础模型，适配多损失优化
    LR_DECAY = 0.95
    WEIGHT_DECAY = 1e-5
    SAVE_INTERVAL = 10
    VAL_INTERVAL = 1
    CHECKPOINT_DIR = "../checkpoints/T5"  # 单独的 checkpoint 目录
    WARMUP_EPOCHS = 5
    GRAD_CLIP = 1.0  # 梯度裁剪防止多损失梯度爆炸
    PATIENCE = 15  # 增加早停耐心，适应复杂损失收敛特性
    MIN_DELTA = 0.001
    GRADIENT_ACCUMULATION_STEPS = 4  # 累积梯度减轻内存压力
    ENABLE_MIXED_PRECISION = True  # 启用混合精度训练加速复杂损失计算

In [2]:
import os
import random
import librosa
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import soundfile as sf
from tqdm import tqdm

class T5NoiseInjector:
    """噪声注入工具类，适配T5模型的复杂损失鲁棒性训练"""
    
    @staticmethod
    def generate_white_noise(length):
        """生成白噪声，符合T5模型输入范围要求"""
        return np.random.randn(length).astype(np.float32)
    
    @staticmethod
    def generate_babble_noise(length, num_speakers=3):
        """生成多说话人混合噪声，增强复杂损失训练的真实性"""
        noise = np.zeros(length, dtype=np.float32)
        for _ in range(num_speakers):
            start = random.randint(0, max(0, length - Config.SAMPLE_RATE))
            end = min(start + Config.SAMPLE_RATE, length)
            noise[start:end] += np.random.randn(end - start).astype(np.float32)
        return noise / num_speakers  # 归一化处理
    
    @staticmethod
    def add_noise(signal, noise_type="white", snr_db=10):
        """按指定SNR添加噪声，确保与T5模型复杂损失计算兼容"""
        if len(signal) == 0:
            return signal
        
        # 计算信号功率，避免数值不稳定
        signal_power = np.mean(signal **2)
        if signal_power < 1e-10:
            return signal
        signal_db = 10 * np.log10(signal_power)
        
        # 根据配置生成指定类型噪声
        if noise_type == "white":
            noise = T5NoiseInjector.generate_white_noise(len(signal))
        elif noise_type == "babble":
            noise = T5NoiseInjector.generate_babble_noise(len(signal))
        else:
            raise ValueError(f"不支持的噪声类型: {noise_type}")
        
        # 调整噪声功率以匹配目标SNR
        noise_power = np.mean(noise** 2)
        noise_db = 10 * np.log10(noise_power + 1e-10)  # 避免log(0)
        target_noise_db = signal_db - snr_db
        noise_scale = 10 ** ((target_noise_db - noise_db) / 20)
        noisy_signal = signal + noise * noise_scale
        
        # 归一化到[-1, 1]范围，确保复杂损失计算稳定性
        max_val = np.max(np.abs(noisy_signal))
        if max_val > 1e-5:
            noisy_signal = noisy_signal / max_val
        return noisy_signal


class T5LibriSpeechDataset(Dataset):
    """T5模型专用数据集，仅加载LibriSpeech开发集数据"""
    
    def __init__(self, split="train", add_noise=False):
        self.split = split
        self.add_noise = add_noise
        self.audio_paths, self.labels = self._load_librispeech_dev()
        self.speaker_to_idx = self._build_speaker_map()
        
        # 调试模式样本量限制
        if Config.DEBUG_MODE:
            if split == "train":
                self.audio_paths = self.audio_paths[:Config.DEBUG_SAMPLE_SIZE]
                self.labels = self.labels[:Config.DEBUG_SAMPLE_SIZE]
            elif split == "val":
                limit = min(Config.DEBUG_SAMPLE_SIZE // 2, len(self.audio_paths))
                self.audio_paths = self.audio_paths[:limit]
                self.labels = self.labels[:limit]
        
        # 验证数据集有效性
        self._validate_dataset()
        print(f"[{split}] 开发集样本数: {len(self.audio_paths)}")
    
    def _load_librispeech_dev(self):
        """仅加载LibriSpeech开发集（含'dev'的子集）"""
        audio_paths = []
        labels = []
        root = Config.LIBRISPEECH_PATH
        
        if not os.path.exists(root):
            raise FileNotFoundError(f"LibriSpeech开发集路径不存在: {root}")
        
        # 仅处理包含'dev'的子目录（开发集）
        for subset in os.listdir(root):
            if "dev" not in subset.lower():
                continue  # 跳过非开发集子集
            
            subset_path = os.path.join(root, subset)
            if not os.path.isdir(subset_path):
                continue
            
            # 遍历开发集子目录结构 (speaker -> chapter -> *.flac)
            for speaker_id in os.listdir(subset_path):
                speaker_path = os.path.join(subset_path, speaker_id)
                if not os.path.isdir(speaker_path):
                    continue
                
                for chapter in os.listdir(speaker_path):
                    chapter_path = os.path.join(speaker_path, chapter)
                    if not os.path.isdir(chapter_path):
                        continue
                    
                    # 收集所有flac文件
                    for file in os.listdir(chapter_path):
                        if file.endswith(".flac"):
                            audio_paths.append(os.path.join(chapter_path, file))
                            labels.append(speaker_id)
        
        # 划分训练/验证集
        if self.split != "test" and len(audio_paths) > 0:
            train_paths, val_paths, train_labels, val_labels = train_test_split(
                audio_paths, labels, 
                test_size=Config.VALID_RATIO, 
                random_state=42,
                stratify=labels  # 保持说话人分布一致
            )
            return (train_paths, train_labels) if self.split == "train" else (val_paths, val_labels)
        
        return audio_paths, labels
    
    def _build_speaker_map(self):
        """构建说话人ID到整数的映射，适配分类损失计算"""
        unique_speakers = sorted(set(self.labels))
        return {speaker: idx for idx, speaker in enumerate(unique_speakers)}
    
    def _validate_dataset(self):
        """验证并移除无效音频文件，确保训练稳定性"""
        invalid_indices = []
        for i in range(len(self.audio_paths)-1, -1, -1):
            path = self.audio_paths[i]
            try:
                if not os.path.exists(path) or os.path.getsize(path) < 1024:
                    raise ValueError("文件不存在或过小")
                # 测试加载文件
                sf.read(path)
            except Exception as e:
                invalid_indices.append(i)
                print(f"移除无效文件: {path}, 原因: {str(e)}")
        
        # 移除无效样本
        for i in invalid_indices:
            self.audio_paths.pop(i)
            self.labels.pop(i)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        """加载并预处理音频样本，返回模型输入格式"""
        path = self.audio_paths[idx]
        label = self.speaker_to_idx[self.labels[idx]]
        
        # 加载音频文件（优先使用soundfile处理flac格式）
        try:
            signal, sr = sf.read(path)
            # 确保采样率匹配
            if sr != Config.SAMPLE_RATE:
                signal = librosa.resample(
                    signal, 
                    orig_sr=sr, 
                    target_sr=Config.SAMPLE_RATE
                )
        except Exception as e:
            print(f"加载失败 {path}，使用静音替代: {e}")
            signal = np.zeros(Config.MAX_SAMPLES, dtype=np.float32)
        
        # 统一音频长度（截断或补零）
        if len(signal) > Config.MAX_SAMPLES:
            signal = signal[:Config.MAX_SAMPLES]
        else:
            signal = np.pad(
                signal, 
                (0, Config.MAX_SAMPLES - len(signal)), 
                mode="constant"
            )
        
        # 归一化处理
        max_val = np.max(np.abs(signal))
        if max_val > 1e-5:
            signal = signal / max_val
        
        # 训练时添加噪声（适配复杂损失的鲁棒性训练）
        if self.add_noise and self.split == "train":
            noise_type = random.choice(Config.NOISE_TYPES)
            snr_db = random.choice(Config.SNR_LEVELS)
            signal = T5NoiseInjector.add_noise(signal, noise_type, snr_db)
        
        # 转换为张量并添加通道维度 [1, MAX_SAMPLES]
        return torch.FloatTensor(signal).unsqueeze(0), label


def get_t5_dataloaders():
    """创建T5模型专用数据加载器"""
    # 实例化数据集
    train_dataset = T5LibriSpeechDataset(split="train", add_noise=True)
    val_dataset = T5LibriSpeechDataset(split="val", add_noise=False)
    test_dataset = T5LibriSpeechDataset(split="test", add_noise=False)
    noisy_test_dataset = T5LibriSpeechDataset(split="test", add_noise=True)
    
    # 检查数据集有效性
    if len(train_dataset) == 0:
        raise RuntimeError("训练集为空，请检查LibriSpeech开发集路径")
    
    # 创建数据加载器
    dataloaders = {
        "train": DataLoader(
            train_dataset,
            batch_size=Config.BATCH_SIZE,
            shuffle=True,
            num_workers=Config.NUM_WORKERS,
            pin_memory=True,
            drop_last=True
        ),
        "val": DataLoader(
            val_dataset,
            batch_size=Config.BATCH_SIZE,
            shuffle=False,
            num_workers=Config.NUM_WORKERS,
            pin_memory=True
        ),
        "test": DataLoader(
            test_dataset,
            batch_size=Config.BATCH_SIZE,
            shuffle=False,
            num_workers=Config.NUM_WORKERS,
            pin_memory=True
        ),
        "noisy_test": DataLoader(
            noisy_test_dataset,
            batch_size=Config.BATCH_SIZE,
            shuffle=False,
            num_workers=Config.NUM_WORKERS,
            pin_memory=True
        )
    }
    
    return dataloaders


# 测试数据加载器
if __name__ == "__main__":
    try:
        dataloaders = get_t5_dataloaders()
        print("数据加载器创建成功")
        
        # 验证训练集批次
        train_loader = dataloaders["train"]
        x, y = next(iter(train_loader))
        print(f"训练集批次形状: 音频 {x.shape} (预期 [batch, 1, 16000]), 标签 {y.shape}")
        print(f"音频值范围: [{x.min():.4f}, {x.max():.4f}]")
        
        # 验证带噪声测试集
        noisy_test_loader = dataloaders["noisy_test"]
        x_noisy, _ = next(iter(noisy_test_loader))
        print(f"带噪声测试集音频范围: [{x_noisy.min():.4f}, {x_noisy.max():.4f}]")
        
    except Exception as e:
        print(f"测试失败: {e}")

[train] 开发集样本数: 5000
[val] 开发集样本数: 557
[test] 开发集样本数: 5567
[test] 开发集样本数: 5567
数据加载器创建成功
训练集批次形状: 音频 torch.Size([16, 1, 16000]) (预期 [batch, 1, 16000]), 标签 torch.Size([16])
音频值范围: [-1.0000, 1.0000]
带噪声测试集音频范围: [-1.0000, 1.0000]


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# 混沌激励模块（增强版，适配复杂损失计算）
class ChaoticStimulus(nn.Module):
    def __init__(self, input_dim, output_dim, chaos_dim=Config.CHAOS_DIM):
        super().__init__()
        self.chaos_transform = nn.Sequential(
            nn.Conv1d(input_dim, chaos_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(chaos_dim),
            nn.PReLU(),
            nn.Conv1d(chaos_dim, output_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(output_dim),
            nn.PReLU()
        )
        
        # 混沌系统参数（用于相位同步损失计算）
        self.chaos_factor = nn.Parameter(torch.tensor(0.1))
        self.chaos_bias = nn.Parameter(torch.randn(1, output_dim, 1))  # 混沌偏置项

    def forward(self, x):
        """
        前向传播，返回处理后的特征和混沌状态（用于复杂损失）
        :param x: 输入特征 [batch_size, channels, seq_len]
        :return: 处理后的特征和混沌状态元组
        """
        # 特征变换
        transformed = self.chaos_transform(x)
        
        # 生成混沌状态（用于相位同步损失）
        chaos_state = torch.sin(transformed + self.chaos_bias) * self.chaos_factor
        
        # 训练时添加混沌扰动
        if self.training:
            transformed = transformed + chaos_state
        
        return transformed, chaos_state


# 改进的注意力机制（支持复杂损失中的权重分析）
class EnhancedAttention(nn.Module):
    def __init__(self, input_dim, heads=Config.ATTENTION_HEADS):
        super().__init__()
        self.heads = heads
        self.head_dim = input_dim // heads
        
        # 注意力线性变换
        self.query = nn.Conv1d(input_dim, input_dim, kernel_size=1)
        self.key = nn.Conv1d(input_dim, input_dim, kernel_size=1)
        self.value = nn.Conv1d(input_dim, input_dim, kernel_size=1)
        self.out = nn.Conv1d(input_dim, input_dim, kernel_size=1)
        
    def forward(self, x):
        """
        前向传播，返回加权特征和注意力权重（用于损失分析）
        :param x: 输入特征 [batch_size, channels, seq_len]
        :return: 加权特征和注意力权重
        """
        batch_size, channels, seq_len = x.shape
        
        # 线性变换并分拆注意力头
        q = self.query(x).view(batch_size, self.heads, self.head_dim, seq_len)  # [B, H, D, L]
        k = self.key(x).view(batch_size, self.heads, self.head_dim, seq_len)    # [B, H, D, L]
        v = self.value(x).view(batch_size, self.heads, self.head_dim, seq_len)  # [B, H, D, L]
        
        # 计算注意力分数 [B, H, L, L]
        scores = torch.matmul(q.transpose(2, 3), k) / (self.head_dim **0.5)
        attn_weights = F.softmax(scores, dim=-1)
        
        # 应用注意力权重
        weighted = torch.matmul(attn_weights, v.transpose(2, 3))  # [B, H, L, D]
        weighted = weighted.transpose(2, 3).contiguous().view(batch_size, channels, seq_len)  # [B, C, L]
        weighted = self.out(weighted)
        
        return weighted, attn_weights


# 统计池化层（保留更多统计特征用于复杂损失）
class StatisticalPooling(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        """
        前向传播，返回更丰富的统计特征
        :param x: 输入特征 [batch_size, channels, seq_len]
        :return: 融合统计特征 [batch_size, channels*4]
        """
        mean = torch.mean(x, dim=2)
        std = torch.std(x, dim=2)
        max_val = torch.max(x, dim=2)[0]
        min_val = torch.min(x, dim=2)[0]
        return torch.cat((mean, std, max_val, min_val), dim=1)


# T5模型的C-HiLAP实现（简化模型+复杂损失适配）
class T5CHiLAPModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.config = Config()
        
        # 特征提取层
        self.feature_extractor = nn.Sequential(
            nn.Conv1d(self.config.INPUT_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU(),
            nn.Conv1d(self.config.HIDDEN_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU(),
            nn.Conv1d(self.config.HIDDEN_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU()
        )
        
        # 混沌激励模块
        self.chaos_layer = ChaoticStimulus(
            self.config.HIDDEN_DIM, 
            self.config.HIDDEN_DIM
        )
        
        # 增强注意力层
        self.attention = EnhancedAttention(
            self.config.HIDDEN_DIM,
            self.config.ATTENTION_HEADS
        )
        
        # TDNN层（深度时序特征提取）
        self.tdnn_block = nn.Sequential(
            nn.Conv1d(self.config.HIDDEN_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=3, dilation=1, padding=1),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU(),
            nn.Conv1d(self.config.HIDDEN_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=3, dilation=2, padding=2),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU(),
            nn.Conv1d(self.config.HIDDEN_DIM, self.config.HIDDEN_DIM, 
                      kernel_size=3, dilation=4, padding=4),
            nn.BatchNorm1d(self.config.HIDDEN_DIM),
            nn.PReLU()
        )
        
        # 池化层
        self.pooling = StatisticalPooling()
        
        # 嵌入层（用于计算李雅普诺夫稳定性损失）
        self.embedding = nn.Sequential(
            nn.Linear(self.config.HIDDEN_DIM * 4, self.config.EMBEDDING_DIM),  # 匹配统计池化输出
            nn.BatchNorm1d(self.config.EMBEDDING_DIM),
            nn.PReLU(),
            nn.Dropout(self.config.DROPOUT_RATE)
        )
        
        # 分类器
        self.classifier = nn.Linear(self.config.EMBEDDING_DIM, num_classes)
        
        # 参考吸引子（用于相位同步损失）
        self.reference_attractor = nn.Parameter(
            torch.randn(1, self.config.HIDDEN_DIM, self.config.CHAOS_TIME_STEPS)
        )

    def forward(self, x):
        """
        前向传播，返回所有复杂损失计算所需的输出
        :param x: 输入音频特征 [batch_size, channels, seq_len]
        :return: 嵌入向量、分类logits、混沌状态、注意力权重（用于损失计算）
        """
        # 调整输入维度
        if x.dim() == 3 and x.size(1) > x.size(2):
            x = x.permute(0, 2, 1)  # 确保形状为 [batch, channels, seq_len]
        
        # 限制序列长度
        seq_len = x.size(2)
        if seq_len > self.config.MAX_SEQ_LEN:
            x = x[:, :, :self.config.MAX_SEQ_LEN]
        
        # 特征提取
        x = self.feature_extractor(x)  # [B, HIDDEN_DIM, L/8]
        
        # 混沌处理（获取混沌状态用于损失）
        x, chaos_state = self.chaos_layer(x)  # [B, HIDDEN_DIM, L/8]
        
        # 注意力处理（获取权重用于损失分析）
        x, attn_weights = self.attention(x)  # [B, HIDDEN_DIM, L/8]
        
        # TDNN处理
        x = self.tdnn_block(x)  # [B, HIDDEN_DIM, L/8]
        
        # 池化
        x = self.pooling(x)  # [B, HIDDEN_DIM*4]
        
        # 嵌入向量（用于李雅普诺夫损失）
        embedding = self.embedding(x)  # [B, EMBEDDING_DIM]
        
        # 分类输出
        logits = self.classifier(embedding)  # [B, num_classes]
        
        # 返回所有复杂损失计算所需组件
        return {
            "embedding": embedding,
            "logits": logits,
            "chaos_state": chaos_state,
            "attn_weights": attn_weights,
            "reference_attractor": self.reference_attractor
        }


# 测试模型
if __name__ == "__main__":
    # 初始化模型（假设分类数为100，根据实际数据集调整）
    num_classes = 100
    model = T5CHiLAPModel(num_classes=num_classes)
    
    # 生成测试输入（匹配LibriSpeech开发集音频格式）
    batch_size = 2
    test_input = torch.randn(batch_size, 1, Config.MAX_SEQ_LEN)  # [batch, channels, seq_len]
    
    print("T5 C-HiLAP模型结构:")
    print(model)
    
    print("\n测试前向传播:")
    print(f"输入形状: {test_input.shape}")
    
    try:
        outputs = model(test_input)
        print(f"嵌入向量形状: {outputs['embedding'].shape}")
        print(f"分类输出形状: {outputs['logits'].shape}")
        print(f"混沌状态形状: {outputs['chaos_state'].shape}")
        print(f"注意力权重形状: {outputs['attn_weights'].shape}")
        print("前向传播成功!")
    except Exception as e:
        print(f"前向传播错误: {e}")
        import traceback
        traceback.print_exc()


T5 C-HiLAP模型结构:
T5CHiLAPModel(
  (feature_extractor): Sequential(
    (0): Conv1d(1, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=1)
    (3): Conv1d(256, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): PReLU(num_parameters=1)
    (6): Conv1d(256, 256, kernel_size=(5,), stride=(2,), padding=(2,))
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): PReLU(num_parameters=1)
  )
  (chaos_layer): ChaoticStimulus(
    (chaos_transform): Sequential(
      (0): Conv1d(256, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): Conv1d(64, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
import gc
import math

# 复杂损失函数实现（T5模型核心）
class ComplexLoss(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.ce_loss = nn.CrossEntropyLoss()
        
    def phase_synchronization_loss(self, chaos_state, reference_attractor):
        """相位同步损失：衡量混沌状态与参考吸引子的同步性"""
        # 调整参考吸引子形状以匹配混沌状态
        batch_size = chaos_state.shape[0]
        ref = reference_attractor.repeat(batch_size, 1, 1)  # [B, C, T]
        
        # 计算动态时间规整(DTW)距离作为同步性度量
        dtw_dist = self._dtw_distance(chaos_state, ref)
        return torch.mean(dtw_dist)
    
    def lyapunov_stability_loss(self, embeddings):
        """李雅普诺夫稳定性损失：衡量嵌入向量的稳定性"""
        # 计算嵌入向量的余弦相似度矩阵
        cos_sim = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
        # 计算相似度方差作为不稳定性度量
        return torch.var(cos_sim)
    
    def _dtw_distance(self, x, y):
        """动态时间规整距离计算（简化版）"""
        batch_size, channels, seq_len = x.shape
        dist_matrix = torch.cdist(x.permute(0, 2, 1), y.permute(0, 2, 1))  # [B, L, T]
        
        # 累积距离计算
        dtw = torch.zeros_like(dist_matrix)
        dtw[:, 0, 0] = dist_matrix[:, 0, 0]
        
        for i in range(1, seq_len):
            dtw[:, i, 0] = dtw[:, i-1, 0] + dist_matrix[:, i, 0]
            
        for j in range(1, y.shape[2]):
            dtw[:, 0, j] = dtw[:, 0, j-1] + dist_matrix[:, 0, j]
            
        for i in range(1, seq_len):
            for j in range(1, y.shape[2]):
                dtw[:, i, j] = dist_matrix[:, i, j] + torch.min(
                    torch.stack([dtw[:, i-1, j], dtw[:, i, j-1], dtw[:, i-1, j-1]]), dim=0
                ).values
                
        return dtw[:, -1, -1]  # 返回每条序列的最终距离
    
    def forward(self, model_outputs, labels, epoch):
        """
        前向传播计算总损失
        :param model_outputs: 模型输出字典
        :param labels: 真实标签
        :param epoch: 当前 epoch（用于损失预热）
        :return: 总损失及各分项损失
        """
        # 基础交叉熵损失
        ce = self.ce_loss(model_outputs["logits"], labels)
        
        # 损失预热：逐步增加复杂损失权重
        warmup_factor = min(1.0, epoch / self.config.LOSS_WARMUP_EPOCHS) if self.training else 1.0
        
        # 相位同步损失
        sync_loss = self.phase_synchronization_loss(
            model_outputs["chaos_state"], 
            model_outputs["reference_attractor"]
        ) * warmup_factor * self.config.SYNC_WEIGHT
        
        # 李雅普诺夫稳定性损失
        lyap_loss = self.lyapunov_stability_loss(model_outputs["embedding"]) * warmup_factor * self.config.LYAPUNOV_WEIGHT
        
        # 总损失
        total_loss = ce * self.config.CE_WEIGHT + sync_loss + lyap_loss
        
        return {
            "total_loss": total_loss,
            "ce_loss": ce,
            "sync_loss": sync_loss,
            "lyap_loss": lyap_loss
        }


# T5模型训练器
class T5Trainer:
    def __init__(self, config=Config, model=None):
        """初始化T5模型训练器"""
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"使用设备: {self.device}")

        # 模型初始化
        if model is None:
            raise ValueError("初始化T5Trainer时必须传入model参数")
        self.model = model.to(self.device)

        # 损失函数和优化器
        self.criterion = ComplexLoss(config)
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=config.LR,
            weight_decay=config.WEIGHT_DECAY
        )

        # 学习率调度器
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3,
        )

        # 学习率预热调度器
        self.warmup_scheduler = optim.lr_scheduler.LambdaLR(
            self.optimizer,
            lr_lambda=lambda epoch: min(1.0, epoch / config.WARMUP_EPOCHS)
        )

        # 混合精度训练
        if config.ENABLE_MIXED_PRECISION and torch.cuda.is_available():
            self.scaler = torch.cuda.amp.GradScaler()
            print("启用混合精度训练")
        else:
            self.scaler = None

        # 检查点目录
        os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)

        # 早停机制
        self.early_stop_counter = 0
        self.best_val_accuracy = 0.0
        self.best_val_loss = float('inf')

    def train_one_epoch(self, dataloader, epoch):
        """训练一个epoch"""
        self.model.train()
        self.criterion.train()
        
        total_metrics = {
            "total_loss": 0.0,
            "ce_loss": 0.0,
            "sync_loss": 0.0,
            "lyap_loss": 0.0,
            "correct": 0,
            "total": 0
        }

        # 梯度累积设置
        accumulation_steps = self.config.GRADIENT_ACCUMULATION_STEPS
        self.optimizer.zero_grad()

        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, (inputs, labels) in progress_bar:
            try:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                # 确保输入维度正确 [batch, 1, seq_len]
                if inputs.dim() == 2:
                    inputs = inputs.unsqueeze(1)

                # 混合精度训练
                if self.scaler is not None:
                    with torch.cuda.amp.autocast():
                        # 模型前向传播
                        outputs = self.model(inputs)
                        # 计算损失
                        losses = self.criterion(outputs, labels, epoch)
                        loss = losses["total_loss"] / accumulation_steps

                    # 反向传播
                    self.scaler.scale(loss).backward()

                    # 梯度裁剪
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(
                        self.model.parameters(), 
                        self.config.GRAD_CLIP
                    )

                    # 梯度累积更新
                    if (i + 1) % accumulation_steps == 0:
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        if epoch <= self.config.WARMUP_EPOCHS:
                            self.warmup_scheduler.step()
                        self.optimizer.zero_grad()

                else:
                    # 标准训练流程
                    outputs = self.model(inputs)
                    losses = self.criterion(outputs, labels, epoch)
                    loss = losses["total_loss"] / accumulation_steps
                    loss.backward()

                    # 梯度裁剪
                    torch.nn.utils.clip_grad_norm_(
                        self.model.parameters(), 
                        self.config.GRAD_CLIP
                    )

                    # 梯度累积更新
                    if (i + 1) % accumulation_steps == 0:
                        self.optimizer.step()
                        if epoch <= self.config.WARMUP_EPOCHS:
                            self.warmup_scheduler.step()
                        self.optimizer.zero_grad()

                # 统计指标
                for key in losses:
                    total_metrics[key] += losses[key].item() * accumulation_steps
                
                # 计算准确率
                _, predicted = outputs["logits"].max(1)
                total_metrics["total"] += labels.size(0)
                total_metrics["correct"] += predicted.eq(labels).sum().item()

                # 更新进度条
                if i % 10 == 0:
                    avg_loss = total_metrics["total_loss"] / (i + 1)
                    accuracy = 100. * total_metrics["correct"] / total_metrics["total"]
                    progress_bar.set_description(
                        f"Epoch {epoch}, Loss: {avg_loss:.4f}, Acc: {accuracy:.2f}%"
                    )

                # 内存管理
                if i % 50 == 0:
                    torch.cuda.empty_cache()
                    gc.collect()

            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"内存不足错误在批次 {i}: {e}")
                    torch.cuda.empty_cache()
                    gc.collect()
                    continue
                else:
                    raise e

        # 计算平均指标
        avg_metrics = {
            key: val / len(dataloader) for key, val in total_metrics.items() if key != "correct" and key != "total"
        }
        avg_metrics["accuracy"] = 100. * total_metrics["correct"] / total_metrics["total"]
        return avg_metrics

    def validate(self, dataloader):
        """验证模型性能"""
        self.model.eval()
        self.criterion.eval()
        
        total_metrics = {
            "total_loss": 0.0,
            "ce_loss": 0.0,
            "sync_loss": 0.0,
            "lyap_loss": 0.0,
            "correct": 0,
            "total": 0
        }

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(dataloader):
                try:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    # 确保输入维度正确
                    if inputs.dim() == 2:
                        inputs = inputs.unsqueeze(1)

                    # 模型前向传播
                    outputs = self.model(inputs)
                    losses = self.criterion(outputs, labels, epoch=1)  # 验证时使用最大权重

                    # 统计损失
                    for key in losses:
                        total_metrics[key] += losses[key].item()

                    # 统计准确率
                    _, predicted = outputs["logits"].max(1)
                    total_metrics["total"] += labels.size(0)
                    total_metrics["correct"] += predicted.eq(labels).sum().item()

                    # 内存管理
                    if i % 20 == 0:
                        torch.cuda.empty_cache()
                        gc.collect()

                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"验证时内存不足: {e}")
                        torch.cuda.empty_cache()
                        gc.collect()
                        continue
                    else:
                        raise e

        # 计算平均指标
        avg_metrics = {
            key: val / len(dataloader) for key, val in total_metrics.items() if key != "correct" and key != "total"
        }
        avg_metrics["accuracy"] = 100. * total_metrics["correct"] / total_metrics["total"]
        return avg_metrics

    def train(self, train_dataloader, val_dataloader):
        """完整训练流程"""
        print("开始T5模型训练...")
        print(f"训练集批次: {len(train_dataloader)}, 验证集批次: {len(val_dataloader)}")

        for epoch in range(1, self.config.EPOCHS + 1):
            try:
                # 训练一个epoch
                train_metrics = self.train_one_epoch(train_dataloader, epoch)
                print(f"Epoch {epoch}/{self.config.EPOCHS}")
                print(f"训练: 总损失={train_metrics['total_loss']:.4f}, "
                      f"交叉熵={train_metrics['ce_loss']:.4f}, "
                      f"同步损失={train_metrics['sync_loss']:.4f}, "
                      f"李雅普诺夫损失={train_metrics['lyap_loss']:.4f}, "
                      f"准确率={train_metrics['accuracy']:.2f}%")

                # 验证
                val_metrics = self.validate(val_dataloader)
                print(f"验证: 总损失={val_metrics['total_loss']:.4f}, "
                      f"准确率={val_metrics['accuracy']:.2f}%")

                # 更新学习率
                self.scheduler.step(val_metrics["total_loss"])
                current_lr = self.optimizer.param_groups[0]['lr']
                print(f"当前学习率: {current_lr:.6f}")

                # 早停检查 (基于验证准确率)
                if val_metrics["accuracy"] > self.best_val_accuracy + self.config.MIN_DELTA:
                    self.best_val_accuracy = val_metrics["accuracy"]
                    self.best_val_loss = val_metrics["total_loss"]
                    self.early_stop_counter = 0
                    # 保存最佳模型
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'val_loss': val_metrics["total_loss"],
                        'val_accuracy': val_metrics["accuracy"]
                    }, os.path.join(self.config.CHECKPOINT_DIR, 'best_t5_model.pth'))
                    print(f"保存最佳模型，验证准确率: {val_metrics['accuracy']:.2f}%")
                else:
                    self.early_stop_counter += 1
                    if self.early_stop_counter >= self.config.PATIENCE:
                        print(f"早停于第 {epoch} 轮")
                        break

                # 定期保存模型
                if epoch % self.config.SAVE_INTERVAL == 0:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'train_loss': train_metrics["total_loss"],
                        'train_accuracy': train_metrics["accuracy"]
                    }, os.path.join(self.config.CHECKPOINT_DIR, f't5_model_epoch_{epoch}.pth'))

                # 清理内存
                torch.cuda.empty_cache()
                gc.collect()

            except KeyboardInterrupt:
                print("训练被用户中断")
                break
            except Exception as e:
                print(f"训练错误: {e}")
                torch.cuda.empty_cache()
                gc.collect()
                continue

        print(f"训练完成，最佳验证准确率: {self.best_val_accuracy:.2f}%")

    def load_checkpoint(self, checkpoint_path):
        """加载检查点"""
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        if 'optimizer_state_dict' in checkpoint:
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint.get('epoch', 0)
        self.best_val_accuracy = checkpoint.get('val_accuracy', self.best_val_accuracy)
        print(f"从第 {epoch} 轮加载检查点，最佳验证准确率: {self.best_val_accuracy:.2f}%")
        return epoch


# 评估器类
class T5Evaluator:
    def __init__(self, model, config=Config):
        """初始化T5模型评估器"""
        self.model = model
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()
        self.criterion = ComplexLoss(config)

    def evaluate(self, dataloader):
        """完整评估，返回所有指标"""
        metrics = {
            "total_loss": 0.0,
            "ce_loss": 0.0,
            "sync_loss": 0.0,
            "lyap_loss": 0.0,
            "accuracy": 0.0,
            "correct": 0,
            "total": 0
        }

        with torch.no_grad():
            for inputs, labels in tqdm(dataloader, desc="评估中"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                if inputs.dim() == 2:
                    inputs = inputs.unsqueeze(1)

                outputs = self.model(inputs)
                losses = self.criterion(outputs, labels, epoch=1)
                
                # 累计损失
                for key in losses:
                    metrics[key] += losses[key].item()
                
                # 累计准确率
                _, predicted = outputs["logits"].max(1)
                metrics["total"] += labels.size(0)
                metrics["correct"] += predicted.eq(labels).sum().item()

        # 计算平均指标
        metrics["accuracy"] = 100. * metrics["correct"] / metrics["total"]
        metrics["total_loss"] /= len(dataloader)
        metrics["ce_loss"] /= len(dataloader)
        metrics["sync_loss"] /= len(dataloader)
        metrics["lyap_loss"] /= len(dataloader)
        
        return metrics


# 测试训练器
if __name__ == "__main__":
    # 加载配置
    config = Config()
    
    # 创建数据加载器
    print("创建T5数据加载器...")
    dataloaders = get_t5_dataloaders()
    
    # 获取类别数
    num_classes = len(dataloaders["train"].dataset.speaker_to_idx)
    print(f"说话人数量（类别数）: {num_classes}")
    
    # 创建模型
    print("初始化T5 C-HiLAP模型...")
    model = T5CHiLAPModel(num_classes=num_classes)
    
    # 创建训练器
    print("初始化训练器...")
    trainer = T5Trainer(config=config, model=model)
    
    # 测试前向传播
    print("测试前向传播...")
    try:
        x, y = next(iter(dataloaders["train"]))
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        
        if x.dim() == 2:
            x = x.unsqueeze(1)
            
        outputs = model(x)
        print(f"嵌入向量形状: {outputs['embedding'].shape}")
        print(f"分类输出形状: {outputs['logits'].shape}")
        
        # 测试损失计算
        losses = trainer.criterion(outputs, y, epoch=1)
        print(f"损失值: {losses['total_loss'].item():.4f}")
        print("前向传播测试成功!")
    except Exception as e:
        print(f"前向传播测试失败: {e}")
        import traceback
        traceback.print_exc()
        exit(1)
    
    # 开始训练
    print("启动训练...")
    trainer.train(dataloaders["train"], dataloaders["val"])
    
    # 评估模型
    print("评估模型性能...")
    evaluator = T5Evaluator(trainer.model, config)
    test_metrics = evaluator.evaluate(dataloaders["test"])
    print("\n测试集性能:")
    print(f"总损失: {test_metrics['total_loss']:.4f}")
    print(f"交叉熵损失: {test_metrics['ce_loss']:.4f}")
    print(f"相位同步损失: {test_metrics['sync_loss']:.4f}")
    print(f"李雅普诺夫损失: {test_metrics['lyap_loss']:.4f}")
    print(f"准确率: {test_metrics['accuracy']:.2f}%")
    
    # 带噪声测试集评估
    noisy_test_metrics = evaluator.evaluate(dataloaders["noisy_test"])
    print("\n带噪声测试集性能:")
    print(f"准确率: {noisy_test_metrics['accuracy']:.2f}%")


创建T5数据加载器...
[train] 开发集样本数: 5000
[val] 开发集样本数: 557
[test] 开发集样本数: 5567
[test] 开发集样本数: 5567
说话人数量（类别数）: 73
初始化T5 C-HiLAP模型...
初始化训练器...
使用设备: cuda
启用混合精度训练
测试前向传播...


/run/nvme/job_5065118/tmp/ipykernel_2107680/3615266546.py:129: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


前向传播测试失败: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 17.38 MiB is free. Process 2070169 has 2.71 GiB memory in use. Process 2073078 has 1.06 GiB memory in use. Process 2183819 has 274.00 MiB memory in use. Process 2184326 has 274.00 MiB memory in use. Process 2186346 has 274.00 MiB memory in use. Process 2186900 has 274.00 MiB memory in use. Including non-PyTorch memory, this process has 974.00 MiB memory in use. Of the allocated memory 851.15 MiB is allocated by PyTorch, and 32.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
启动训练...
开始T5模型训练...
训练集批次: 312, 验证集批次: 35


Traceback (most recent call last):
  File "/run/nvme/job_5065118/tmp/ipykernel_2107680/3615266546.py", line 477, in <module>
    outputs = model(x)
              ^^^^^^^^
  File "/usr/local/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/run/nvme/job_5065118/tmp/ipykernel_2107680/2230600108.py", line 186, in forward
    x, chaos_state = self.chaos_layer(x)  # [B, HIDDEN_DIM, L/8]
                     ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.12/site-package

内存不足错误在批次 0: CUDA out of memory. Tried to allocate 490.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 259.38 MiB is free. Process 2070169 has 2.71 GiB memory in use. Process 2073078 has 1.06 GiB memory in use. Process 2183819 has 274.00 MiB memory in use. Process 2184326 has 274.00 MiB memory in use. Process 2186346 has 274.00 MiB memory in use. Process 2186900 has 274.00 MiB memory in use. Including non-PyTorch memory, this process has 732.00 MiB memory in use. Of the allocated memory 606.34 MiB is allocated by PyTorch, and 27.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
内存不足错误在批次 1: CUDA out of memory. Tried to allocate 490.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 259.38 MiB is free. Process 2070169 has 2.71 GiB memory 

...

内存不足错误在批次 15: CUDA out of memory. Tried to allocate 490.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 271.38 MiB is free. Process 2070169 has 2.71 GiB memory in use. Process 2073078 has 1.06 GiB memory in use. Process 2183819 has 274.00 MiB memory in use. Process 2184326 has 274.00 MiB memory in use. Process 2186346 has 274.00 MiB memory in use. Process 2186900 has 274.00 MiB memory in use. Including non-PyTorch memory, this process has 720.00 MiB memory in use. Of the allocated memory 606.47 MiB is allocated by PyTorch, and 15.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
内存不足错误在批次 16: CUDA out of memory. Tried to allocate 490.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 271.38 MiB is free. Process 2070169 has 2.71 GiB memor

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fffb51edb20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  6%|▌         | 18/312 [00:03<01:02,  4.73it/s]


训练被用户中断
训练完成，最佳验证准确率: 0.00%
评估模型性能...


评估中:   0%|          | 0/348 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacity of 4.75 GiB of which 675.38 MiB is free. Process 2070169 has 2.71 GiB memory in use. Process 2073078 has 1.06 GiB memory in use. Process 2183819 has 274.00 MiB memory in use. Process 2184326 has 274.00 MiB memory in use. Process 2186346 has 274.00 MiB memory in use. Process 2186900 has 274.00 MiB memory in use. Including non-PyTorch memory, this process has 316.00 MiB memory in use. Of the allocated memory 164.93 MiB is allocated by PyTorch, and 53.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)